### 7. Сверточные нейронные сети для анализа текста

#### Задание<br>
Берем отызывы за лето (из архива с материалами или предыдущего занятия)<br>
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек:
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше



#### Загрузим необходимые библиотеки и данные

In [3]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=9c01e4ce0f66eb6a6ce274b0134d4b396cfb1d5056410c84fae50b0e4a8d804e
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [4]:
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.2 MB/s eta 0:00:00


In [5]:
!pip install keras_mixed_sequence

  Preparing metadata (setup.py) ... done
  Created wheel for keras_mixed_sequence: filename=keras_mixed_sequence-1.0.28-py3-none-any.whl size=7509 sha256=888fe6395c397b57684f1e82f538e154ec4b3809e42d925704715581dca6cb00
  Stored in directory: /root/.cache/pip/wheels/02/d9/91/a8d0cf2cb135ba88d53fd3bb713e3499cea48552bb8a8b5de3
Successfully built keras_mixed_sequence


In [6]:
!pip install tensorflow

In [7]:
!pip install tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.3 MB/s eta 0:00:00


In [8]:
import gensim
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy3 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
import tokenizer
#from keras_mixed_sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder

from pathlib import Path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
# Функция подсчета f1_score
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [10]:
!pip install xlrd

In [11]:
from google.colab import files

In [12]:
uploaded = files.upload()

Saving Отзывы.xls to Отзывы.xls


In [13]:
df = pd.read_excel('Отзывы.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [14]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [15]:
#разобъем выборку на test/train
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [16]:
df_test

,Rating,Content,Date
0,5,Всё отлично!!!,2017-08-11
1,5,Мне нравится. Удобно. Еще ни разу не подвел!,2017-06-19
2,3,Иногда слишком долго приходится ждать пока зап...,2017-08-10
3,5,Отличное приложение. Легко отслеживать свои ра...,2017-07-22
4,5,Все грамотно,2017-07-06
...,...,...,...
6813,5,Здравствуйте сделайте пожалуйста чтоб я мог по...,2017-07-12
6814,5,Все хорошо,2017-08-01
6815,5,"Отличное приложение. Удобно, быстро.",2017-08-02
6816,1,Уберите или исправьте антивирус. Жалуется на L...,2017-08-02


In [17]:
df_train

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08
...,...,...,...
13836,4,Все нравится,2017-07-29
13837,5,Очень смешно программа пугается рута :),2017-07-28
13838,1,Не могу скачать ошибка номер 24,2017-08-06
13839,5,Сбербанк всегда рядом,2017-08-12


In [18]:
import inspect
import pymorphy3
assert True

In [19]:
#подготовка данных
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = pymorphy3.MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [20]:
#обработка корпуса
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [21]:
#фильтр токенов и словарь по количеству самых популярных
tokens_filtered = [word for word in tokens if word.isalnum()]

In [22]:
dist = FreqDist(tokens_filtered)
dist

FreqDist({'приложение': 4123, 'удобно': 2201, 'работать': 1288, 'удобный': 1182, 'отлично': 860, 'нравиться': 763, 'хороший': 681, 'отличный': 677, 'телефон': 627, 'супер': 540, ...})

In [23]:
max_words = 100

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [24]:
#словарь из топ-100 токенов, включая рейтинг
glossary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
glossary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'хороший': 7,
 'отличный': 8,
 'телефон': 9,
 'супер': 10,
 'быстро': 11,
 'обновление': 12,
 'пароль': 13,
 'мочь': 14,
 'пользоваться': 15,
 'антивирус': 16,
 'банк': 17,
 'вход': 18,
 'устраивать': 19,
 'сбербанк': 20,
 'раз': 21,
 'прошивка': 22,
 'карта': 23,
 'проблема': 24,
 'рута': 25,
 'программа': 26,
 'ошибка': 27,
 'разработчик': 28,
 'сделать': 29,
 'приходиться': 30,
 'вводить': 31,
 'перевод': 32,
 'счёт': 33,
 'писать': 34,
 'норма': 35,
 'деньга': 36,
 'довольный': 37,
 'около': 38,
 'постоянно': 39,
 'нормально': 40,
 'код': 41,
 'исправить': 42,
 'смс': 43,
 'платёж': 44,
 'понятно': 45,
 'последний': 46,
 'функция': 47,
 'зайти': 48,
 'свой': 49,
 'вылетать': 50,
 'мобильный': 51,
 'стать': 52,
 'шаблон': 53,
 'приходить': 54,
 'возможность': 55,
 'право': 56,
 'делать': 57,
 'иня': 58,
 'проверка': 59,
 'класс': 60,
 'установить': 61,
 'root': 62,
 'заходить': 63,
 '5': 

Подготовим данные к обучению:
переводим в нижний регистр, токенизируем, удаляем символы, проверяем на присутствие в нашем словаре, добавляем (если +), выравниваем нулями по длине

In [26]:
#подготовка к обучению

max_len = 20

def text_to_sequence(text, max_len):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in glossary:
            result.append(glossary[word])
    padding = [0]*(max_len-len(result))
    return padding + result[-max_len:]

In [27]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[ 0,  0,  0, ...,  1,  2, 15],
       [ 0,  0,  0, ...,  0,  0,  2],
       [ 0,  0,  0, ...,  0,  0,  5],
       ...,
       [ 0,  0,  0, ..., 14, 27, 84],
       [ 0,  0,  0, ...,  0,  0, 20],
       [ 0,  0,  0, ...,  0,  0,  5]], dtype=int32)

In [28]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating'])
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [29]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [30]:
num_classes = 5 #количество оценок рейтинга
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

###  Инициализируем и обучим модель Keras CONV с Embedding слоем по умолчанию

In [31]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, inputlen), на выходе: (батч, inputlen, outputdim)
model.add(Conv1D(128, 3))
                   #128 -- длина 1D-фильтра, шаг -- 3
                   #на выходе ([128/3], 128)
model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя
                    #на выходе ([128/3], 128)
model.add(GlobalMaxPool1D())
                    #в каждой свертке оставляет максимальный элемент
                    #на выходе ([128/3], 1)
model.add(Dense(10))
               #10-количество выходов
model.add(Activation("relu"))
model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [32]:
#компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [33]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [34]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 11s 45ms/step - loss: 1.1708 - get_f1: 0.3681 - val_loss: 0.9915 - val_get_f1: 0.7003
Epoch 2/20
25/25 [==============================] - 0s 15ms/step - loss: 0.8869 - get_f1: 0.7240 - val_loss: 0.8054 - val_get_f1: 0.7357
Epoch 3/20
25/25 [==============================] - 0s 15ms/step - loss: 0.7585 - get_f1: 0.7467 - val_loss: 0.7505 - val_get_f1: 0.7450
Epoch 4/20
25/25 [==============================] - 0s 15ms/step - loss: 0.7239 - get_f1: 0.7509 - val_loss: 0.7417 - val_get_f1: 0.7440
Epoch 5/20
25/25 [==============================] - 1s 24ms/step - loss: 0.7087 - get_f1: 0.7501 - val_loss: 0.7364 - val_get_f1: 0.7437
Epoch 6/20
25/25 [==============================] - 1s 25ms/step - loss: 0.6960 - get_f1: 0.7521 - val_loss: 0.7319 - val_get_f1: 0.7430
Epoch 7/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6841 - get_f1: 0.7510 - val_loss: 0.7295 - val_get_f1: 0.7449
Epoch 8/20
25/25 [=======================

In [71]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', round(score[0],2))
print('Test f1_score:', round(score[1],2))

14/14 [==============================] - 0s 3ms/step - loss: 0.7501 - get_f1: 0.7555


Test loss: 0.75
Test f1_score: 0.76


### Инициализируем и обучим модель Keras CONV с предобученным Embedding слоем

In [36]:
!wget http://vectors.nlpl.eu/repository/20/180.zip

--2023-10-17 07:51:18--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  23.1MB/s    in 21s     

2023-10-17 07:51:40 (21.8 MB/s) - ‘180.zip’ saved [484452317/484452317]



In [37]:
!unzip 180.zip

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [38]:
#Загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/model.bin', binary=True) .vectors

In [48]:
max_len = 20

def text_to_sequence(text, max_len):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in glossary:
            result.append(glossary[word])
    padding = [0]*(max_len-len(result))
    return padding + result[-max_len:]

In [49]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_train

array([[ 0,  0,  0, ...,  1,  2, 15],
       [ 0,  0,  0, ...,  0,  0,  2],
       [ 0,  0,  0, ...,  0,  0,  5],
       ...,
       [ 0,  0,  0, ..., 14, 27, 84],
       [ 0,  0,  0, ...,  0,  0, 20],
       [ 0,  0,  0, ...,  0,  0,  5]], dtype=int32)

In [50]:
num_classes = 5 #количество оценок рейтинга
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [58]:
#инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [59]:
#компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [60]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 3s 46ms/step - loss: 1.3069 - get_f1: 0.5226 - val_loss: 0.9547 - val_get_f1: 0.7072
Epoch 2/20
25/25 [==============================] - 0s 9ms/step - loss: 0.8533 - get_f1: 0.7308 - val_loss: 0.8379 - val_get_f1: 0.7291
Epoch 3/20
25/25 [==============================] - 0s 14ms/step - loss: 0.7868 - get_f1: 0.7430 - val_loss: 0.8163 - val_get_f1: 0.7377
Epoch 4/20
25/25 [==============================] - 0s 9ms/step - loss: 0.7536 - get_f1: 0.7530 - val_loss: 0.8074 - val_get_f1: 0.7323
Epoch 5/20
25/25 [==============================] - 1s 24ms/step - loss: 0.7337 - get_f1: 0.7561 - val_loss: 0.7868 - val_get_f1: 0.7396
Epoch 6/20
25/25 [==============================] - 1s 24ms/step - loss: 0.7121 - get_f1: 0.7644 - val_loss: 0.7912 - val_get_f1: 0.7369


In [69]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', round(score[0],2))
print('Test f1_score:', round(score[1],4))

14/14 [==============================] - 0s 3ms/step - loss: 0.7501 - get_f1: 0.7555


Test loss: 0.75
Test f1_score: 0.7555


### Что получилось:

Модель Keras CONV без инициализации предобученных embedings дает лучшую метрику (f1_score = 0.76), чем предобученная модель (f1_score = 0.75). Результаты очень близки на этих данных, возможно дело в предобработке начального текста, либо в размерности самих предобученных и сырых эмбедингов.